In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# One cell RNN input_dim (3) -> output_dim (5)
hidden_size = 5
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
print(cell.output_size, cell.state_size)

x_data = np.array([[[1, 2, 3]]], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

5 5
array([[[-0.47360685, -0.49077907, -0.59324473, -0.80968446,  0.95318228]]], dtype=float32)


In [3]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# One cell RNN input_dim (3) -> output_dim (5). sequence: 2
hidden_size = 5
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]]], dtype=np.float32)
outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[-0.66892838, -0.90978724,  0.99134082, -0.19908188, -0.69907773],
        [-0.82586318, -0.99585074,  0.99999833, -0.9550944 , -0.59060866]]], dtype=float32)


In [4]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# One cell RNN input_dim (3) -> output_dim (5). sequence: 2
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]],

                   [[7, 8, 9],
                    [10, 11, 12]],

                   [[13, 14, 15],
                    [16, 17, 18]], ], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_data, sequence_length=[1,2,1], dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ -2.56089836e-01,   1.71408668e-01,  -1.45856142e-01,
           3.50203030e-02,   2.01694384e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -4.78336141e-02,   2.44351238e-01,  -2.97077894e-01,
           1.80373201e-03,   4.38719481e-01],
        [ -6.20230287e-02,   2.24532843e-01,  -3.92552167e-01,
           2.00625160e-03,   7.10222661e-01]],

       [[ -4.94097546e-03,   1.87834799e-01,  -3.56861621e-01,
           2.20259790e-05,   5.88683724e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# One cell RNN input_dim (3) -> output_dim (5). sequence: 2

batch_size = 3
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)

x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]],

                   [[7, 8, 9],
                    [10, 11, 12]],

                   [[13, 14, 15],
                    [16, 17, 18]], ], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                     initial_state=initial_state, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[  4.84392643e-02,  -3.13782059e-02,   8.60535204e-02,
          -4.40735444e-02,   2.98324555e-01],
        [  2.71116942e-03,  -1.85060443e-03,   2.88982481e-01,
          -6.42292295e-03,   5.30039310e-01]],

       [[  1.94126387e-05,   1.76166609e-01,   2.34703779e-01,
          -4.92287276e-04,   1.10681273e-01],
        [  3.20592676e-07,   3.98451626e-01,   4.95123982e-01,
          -4.43883036e-05,   1.57826379e-01]],

       [[  2.79868351e-09,   3.71313363e-01,   2.49200687e-01,
          -3.77055744e-06,   2.09293943e-02],
        [  3.95419981e-11,   5.67828178e-01,   4.92595583e-01,
          -3.39440106e-07,   2.89667398e-02]]], dtype=float32)


In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# Create input data
x_data = np.arange(24, dtype=np.float32).reshape(2, 4, 3)
pp.pprint(x_data)  # batch, sequence_length, input size

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.]],

       [[ 12.,  13.,  14.],
        [ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.]]], dtype=float32)


In [7]:
# Make rnn
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True)

# rnn in/out
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
print("dynamic rnn: ", outputs)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("rnn/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[-0.00034569, -0.00070532,  0.00043568,  0.00069213,  0.00085086],
        [ 0.00170664, -0.00082707,  0.00185418, -0.00077598, -0.00035609],
        [ 0.00705891,  0.00040856,  0.00350947, -0.00549153, -0.00428402],
        [ 0.01491776,  0.00288426,  0.0046613 , -0.01214804, -0.00930375]],

       [[ 0.0066977 ,  0.00251319,  0.00212484, -0.00621179, -0.00586229],
        [ 0.01622089,  0.00602045,  0.0038038 , -0.01418692, -0.01209893],
        [ 0.02632577,  0.01001449,  0.00443646, -0.02170778, -0.01691542],
        [ 0.0357908 ,  0.01427082,  0.00417097, -0.02784121, -0.02004039]]], dtype=float32)


In [8]:
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                     sequence_length=[1, 2])
print("dynamic rnn: ", outputs)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size
sess.close()

dynamic rnn:  Tensor("rnn_1/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[  2.08606524e-03,  -3.55264395e-02,   8.31993893e-02,
          -3.29029299e-02,   4.88560162e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  6.47814631e-01,  -1.61745623e-01,   6.02805929e-04,
          -6.00308599e-03,   2.57990300e-03],
        [  7.89284527e-01,  -3.44720364e-01,   5.33071987e-04,
          -3.10120126e-03,   6.17182348e-04],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# bi-directional rnn
cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                  sequence_length=[2, 3],
                                                  dtype=tf.float32)

sess.run(tf.global_variables_initializer())
pp.pprint(sess.run(outputs))
pp.pprint(sess.run(states))

(   array([[[  2.30417088e-01,  -1.56091116e-02,  -1.05227932e-01,
           1.00008920e-01,  -6.87071159e-02],
        [  3.88323992e-01,  -2.48035818e-01,  -2.34708935e-01,
           1.02263860e-01,  -1.30803376e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  1.96846272e-03,  -1.28378838e-01,  -7.41831213e-02,
           2.21356186e-06,  -4.30520810e-02],
        [  2.42746691e-03,  -8.88744667e-02,  -5.13007529e-02,
           2.25100734e-06,  -3.24536823e-02],
        [  2.52628908e-03,  -6.03847019e-02,  -2.98893023e-02,
           2.18062837e-06,  -1.97081175e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32),
    array([[[ 0.52327591, -0.59075385, -0.46282047,  0.47981259, -0.45508054],
        [ 0.45975351, -0.

In [10]:
# Broadcasting based softmax
softmax_w = np.arange(12, dtype=np.float32).reshape(4, 3)
outputs = x_data * softmax_w
pp.pprint(softmax_w)
pp.pprint(outputs)
outputs = x_data * softmax_w + [1, 2, 3]
pp.pprint(outputs)

array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.]], dtype=float32)
array([[[   0.,    1.,    4.],
        [   9.,   16.,   25.],
        [  36.,   49.,   64.],
        [  81.,  100.,  121.]],

       [[   0.,   13.,   28.],
        [  45.,   64.,   85.],
        [ 108.,  133.,  160.],
        [ 189.,  220.,  253.]]], dtype=float32)
array([[[   1.,    3.,    7.],
        [  10.,   18.,   28.],
        [  37.,   51.,   67.],
        [  82.,  102.,  124.]],

       [[   1.,   15.,   31.],
        [  46.,   66.,   88.],
        [ 109.,  135.,  163.],
        [ 190.,  222.,  256.]]])


In [11]:
# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
prediction2 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction3 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval())

Loss1:  0.313262 Loss2:  1.31326 Loss3:  0.646595
